# Mise en place d’un système de recommandation basé sur un autoencodeur

### Partie 1 : Préparation des données
Chargement et exploration des données
- Chargez le fichier bbc-text.csv en utilisant pandas.
- Affichez les cinq premières lignes du DataFrame.
- Expliquez le rôle des colonnes category et text.

In [9]:
import pandas as pd

# Chargement du fichier CSV
df = pd.read_csv('bbc-new.csv')

# Affichage des cinq premières lignes du DataFrame
print(df.head())

# Explication des colonnes
# La colonne 'category' représente la catégorie ou le sujet de l'article.
# La colonne 'text' contient le texte de l'article.

        category                                               text
0           tech  tv future in the hands of viewers with home th...
1       business  worldcom boss  left books alone  former worldc...
2          sport  tigers wary of farrell  gamble  leicester say ...
3          sport  yeading face newcastle in fa cup premiership s...
4  entertainment  ocean s twelve raids box office ocean s twelve...


#### Encodage des catégories
- Encodez les valeurs de la colonne category en utilisant LabelEncoder pour obtenir des
valeurs numériques.
- Ajoutez une colonne category_encoded au DataFrame contenant les catégories encodées.
- Affichez les catégories encodées et leur correspondance avec les catégories d’origine.

In [23]:
from sklearn.preprocessing import LabelEncoder

# Initialisation du LabelEncoder
label_encoder = LabelEncoder()

# Encodage des catégories
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Affichage des catégories encodées et leur correspondance
category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Catégories encodées et leur correspondance :")
print(category_mapping)

# Affichage des cinq premières lignes du DataFrame avec la nouvelle colonne
print(df.head())


Catégories encodées et leur correspondance :
{'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}
        category                                               text  \
0           tech  tv future in the hands of viewers with home th...   
1       business  worldcom boss  left books alone  former worldc...   
2          sport  tigers wary of farrell  gamble  leicester say ...   
3          sport  yeading face newcastle in fa cup premiership s...   
4  entertainment  ocean s twelve raids box office ocean s twelve...   

   category_encoded  
0                 4  
1                 0  
2                 3  
3                 3  
4                 1  


#### Vectorisation TF-IDF
- Utilisez TfidfVectorizer pour transformer la colonne text en vecteurs numériques.
- Limitez le nombre de caractéristiques à 5000 et appliquez un prétraitement pour ignorer
les mots vides (stop words).
- Affichez la forme de la matrice obtenue.

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialisation du TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

# Transformation de la colonne text
tfidf_matrix = tfidf_vectorizer.fit_transform(df['text'])

# Affichage de la forme de la matrice obtenue
print("Forme de la matrice TF-IDF :", tfidf_matrix.shape)


Forme de la matrice TF-IDF : (2225, 5000)


#### Séparation des données
- Séparez les données en ensembles d’entraînement et de test avec un ratio 80%-20%.
- Expliquez l’importance de diviser les données en deux ensembles.


In [27]:
from sklearn.model_selection import train_test_split

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, df['category_encoded'], test_size=0.2, random_state=42)

# Affichage des tailles des ensembles d'entraînement et de test
print("Taille de l'ensemble d'entraînement :", X_train.shape)
print("Taille de l'ensemble de test :", X_test.shape)


Taille de l'ensemble d'entraînement : (1780, 5000)
Taille de l'ensemble de test : (445, 5000)


### Partie 2 : Construction et entraînement de l’autoencodeur

Définition de l’architecture de l’autoencodeur
- Définissez les dimensions d’entrée et de sortie du modèle en fonction des données d’entrée.
- Construisez un modèle avec :
- Des couches encodées de dimensions décroissantes (512, 256, 128).
- Des couches décodées de dimensions croissantes (128, 256, 512) pour reconstituer les
données d’entrée.
- Utilisez relu comme fonction d’activation dans les couches cachées et sigmoid dans la
couche de sortie.


In [37]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Définir les dimensions d'entrée et de sortie
input_dim = tfidf_matrix.shape[1]

# Définir l'entrée de l'autoencodeur
input_layer = Input(shape=(input_dim,))

# Construire les couches encodées
encoded = Dense(512, activation='relu')(input_layer)
encoded = Dense(256, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)

# Construire les couches décodées
decoded = Dense(256, activation='relu')(encoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Définir le modèle de l'autoencodeur
autoencoder = Model(input_layer, decoded)

# Compiler le modèle
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Afficher l'architecture du modèle
autoencoder.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 5000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,560,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5000)           │     2,565,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,454,344 (20.81 MB)

 Trainable params: 5,454,344 (20.81 MB)

 Non-trainable params: 0 (0.00 B)

#### Compilation et entraînement
- Compilez le modèle en utilisant l’optimiseur Adam avec un taux d’apprentissage de 0,001
et une fonction de perte mse.
- Entraînez l’autoencodeur sur les données d’entraînement en utilisant 50 époques et une
taille de lot de 128.
- Évaluez les performances du modèle sur les données de test.

In [44]:
import numpy as np

# Convertir les matrices creuses en matrices denses
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Compiler le modèle
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Entraîner l'autoencodeur
history = autoencoder.fit(X_train_dense, X_train_dense,
                          epochs=50,
                          batch_size=128,
                          shuffle=True,
                          validation_data=(X_test_dense, X_test_dense))

# Évaluer les performances du modèle sur les données de test
loss = autoencoder.evaluate(X_test_dense, X_test_dense)
print(f"Perte sur les données de test : {loss}")


Epoch 1/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - loss: 0.2455 - val_loss: 0.1566
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.0607 - val_loss: 2.0000e-04
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 2.0000e-04 - val_loss: 2.0000e-04
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - loss: 2.0000e-04 - val_lo

### Partie 3 : Évaluation du modèle
Calcul de l’erreur
- Implémentez une fonction pour calculer la racine de l’erreur quadratique moyenne (RMSE)
entre les données originales et reconstruites.
- Calculez le RMSE sur les ensembles d’entraînement et de test.
- Interprétez les résultats obtenus.

In [49]:
import numpy as np
from sklearn.metrics import mean_squared_error

# Fonction pour calculer le RMSE
def calculate_rmse(original, reconstructed):
    mse = mean_squared_error(original, reconstructed)
    rmse = np.sqrt(mse)
    return rmse

# Reconstruire les données d'entraînement et de test
X_train_reconstructed = autoencoder.predict(X_train_dense)
X_test_reconstructed = autoencoder.predict(X_test_dense)

# Calculer le RMSE sur les ensembles d'entraînement et de test
rmse_train = calculate_rmse(X_train_dense, X_train_reconstructed)
rmse_test = calculate_rmse(X_test_dense, X_test_reconstructed)

print(f"RMSE sur l'ensemble d'entraînement : {rmse_train}")
print(f"RMSE sur l'ensemble de test : {rmse_test}")


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
RMSE sur l'ensemble d'entraînement : 0.01414213562373095
RMSE sur l'ensemble de test : 0.01414213562373095


### Partie 4 : Système de recommandation
Extraction des représentations encodées
- Utilisez le modèle encodeur pour générer des représentations compactes des textes.
- Affichez quelques exemples de ces représentations.


In [52]:
from tensorflow.keras.models import Model

# Extraire le modèle encodeur
encoded_input = Input(shape=(input_dim,))
encoder_layers = autoencoder.layers[1:4]  # Les trois premières couches sont les couches de l'encodeur
encoded_output = encoded_input
for layer in encoder_layers:
    encoded_output = layer(encoded_output)

encoder = Model(encoded_input, encoded_output)

# Générer des représentations compactes
encoded_train = encoder.predict(X_train_dense)
encoded_test = encoder.predict(X_test_dense)

# Afficher quelques exemples de ces représentations
print("Représentations compactes de l'ensemble d'entraînement :")
print(encoded_train[:5])  # Afficher les 5 premières représentations de l'ensemble d'entraînement

print("Représentations compactes de l'ensemble de test :")
print(encoded_test[:5])  # Afficher les 5 premières représentations de l'ensemble de test


56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Représentations compactes de l'ensemble d'entraînement :
[[0.         0.         0.         0.         0.         0.
  0.         3.568185   0.         1.4984218  0.         0.
  0.         5.6321926  4.4118915  6.401987   0.         0.
  2.095706   3.5047462  0.09026669 4.5948005  2.835757   3.099318
  0.         0.         3.95749    0.         3.7217495  0.
  4.454132   0.         1.6529427  0.         0.         0.
  3.7038362  4.297579   3.1414104  0.12558293 0.         0.
  4.2598333  0.         1.6419827  0.         5.8168144  0.
  3.4232147  2.4404862  3.6500404  0.         0.         4.772605
  7.260083   4.833711   3.4349132  4.38844    0.40744755 4.619599
  2.6899898  1.1154891  4.030833   0.         0.         4.874888
  3.0669496  5.232492   2.8698444  4.0793185  3.8814511  0.
  6.2507267  0.         1.6912237  3.1970408  4.521489   0.
  5.1938434  0.         2.592547   0.         4.451839   5.3

#### Implémentation d’une fonction de recommandation
- Créez une fonction recommend qui prend en entrée :
- Un texte d’utilisateur.
- Le nombre de recommandations souhaitées (par défaut : 5).

Cette fonction doit :
- Transformer le texte d’entrée en vecteur TF-IDF.
- Extraire sa représentation encodée.
- Calculer la similarité entre cette représentation et celles des textes du dataset.
- Retourner les textes les plus similaires

In [56]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Fonction de recommandation
def recommend(user_text, num_recommendations=5):
    # Transformer le texte d'entrée en vecteur TF-IDF
    user_tfidf = tfidf_vectorizer.transform([user_text])

    # Extraire la représentation encodée du texte d'entrée
    user_encoded = encoder.predict(user_tfidf.toarray())

    # Calculer la similarité entre la représentation encodée du texte d'entrée et celles des textes du dataset
    similarities = cosine_similarity(user_encoded, encoded_train)

    # Obtenir les indices des textes les plus similaires
    similar_indices = similarities.argsort()[0][-num_recommendations:][::-1]

    # Retourner les textes les plus similaires
    recommended_texts = df.iloc[similar_indices]['text'].tolist()
    return recommended_texts

# Exemple d'utilisation de la fonction de recommandation
user_text = "Votre texte d'entrée ici"
recommendations = recommend(user_text, num_recommendations=5)
print("Textes recommandés :")
for text in recommendations:
    print(text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Textes recommandés :
borussia dortmund near bust german football club and former european champion borussia dortmund has warned it will go bankrupt if rescue talks with creditors fail.  the company s shares tumbled after it said it has  entered a life-threatening profitability and financial situation . borussia dortmund has posted record losses and missed rent payments on its westfallen stadium. chief executive gerd niebaum stepped down last week and creditors are now pushing for greater control. shares in borussia dortmund  germany s only stock-market listed football club  dropped by almost 23% to 2.05 euros during early afternoon trading.  fund manager florian hamm - borussia dortmund s largest investor - said he would only invest more money in the company if he got a greater say in how it is run.  i demand better transparency   he is quoted as saying by germany s manger magazin. the club has also faced calls to appoint executives from outside th

#### Test de la fonction de recommandation
- Testez la fonction avec un texte exemple (par exemple : The stock market shows significant
growth).
- Affichez les recommandations obtenues, avec leurs catégories et contenus.

In [63]:
# Définir un texte exemple
user_text = "The stock market shows significant growth"

# Appeler la fonction de recommandation
recommendations = recommend(user_text, num_recommendations=5)

# Afficher les recommandations obtenues avec leurs catégories et contenus
print("Textes recommandés :")
for text in recommendations:
    # Trouver l'index du texte recommandé dans le DataFrame
    index = df[df['text'] == text].index[0]
    category = df.loc[index, 'category']
    content = df.loc[index, 'text']
    print(f"Catégorie : {category}")
    print(f"Contenu : {content}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Textes recommandés :
Catégorie : entertainment
Contenu : prince crowned  top music earner  prince earned more than any other pop star in 2004  beating artists such madonna and elton john in us magazine rolling stone s annual list.  the singer banked $56.5m (£30.4m) from concerts  album and publishing sales with his musicology tour and album. he kept madonna in second place  as she earned $54.9m (£29.5m) while embarking on her global re-invention tour. veterans simon and garfunkel were in 10th place  their comeback tour helping them earn $24.9m (£13.4m) last year.   prince returned to centre stage after a decade in the commercial wilderness   the magazine reported. the singer s 2004 tour took $90.3m (£48.5m) in ticket sales and he sold 1.9 million copies of his latest album musicology.  although she grossed more than prince last year  madonna remained in second place because of the  monumental  production costs of her tour. heavy metal band metalli

# Rapport Technique : Construction et Évaluation d'un Autoencodeur pour la Recommandation de Textes

## Introduction

Ce rapport technique présente les étapes de construction, d'entraînement et d'évaluation d'un autoencodeur pour la recommandation de textes. Nous avons utilisé un ensemble de données de textes pour entraîner l'autoencodeur et générer des représentations compactes des textes. Ensuite, nous avons implémenté un système de recommandation basé sur ces représentations.

## Partie 1 : Préparation des Données

### Chargement et Exploration des Données

Nous avons chargé le fichier `bbc-text.csv` en utilisant pandas et exploré les cinq premières lignes du DataFrame. Les colonnes `category` et `text` ont été expliquées :

- **category** : Représente la catégorie ou le sujet de l'article.
- **text** : Contient le texte complet de l'article.

### Encodage des Catégories

Nous avons utilisé `LabelEncoder` pour encoder les valeurs de la colonne `category` en valeurs numériques et ajouté une nouvelle colonne `category_encoded` au DataFrame.

### Vectorisation TF-IDF

Nous avons utilisé `TfidfVectorizer` pour transformer la colonne `text` en vecteurs numériques, en limitant le nombre de caractéristiques à 5000 et en ignorant les mots vides (stop words). La forme de la matrice TF-IDF obtenue a été affichée.

## Partie 2 : Construction et Entraînement de l'Autoencodeur

### Définition de l'Architecture de l'Autoencodeur

Nous avons défini un autoencodeur avec les dimensions d'entrée et de sortie basées sur les données d'entrée. L'architecture de l'autoencodeur comprend :

- **Couches encodées** : Dimensions décroissantes (512, 256, 128).
- **Couches décodées** : Dimensions croissantes (128, 256, 512) pour reconstituer les données d'entrée.
- **Fonctions d'activation** : `relu` dans les couches cachées et `sigmoid` dans la couche de sortie.

### Compilation et Entraînement

Nous avons compilé le modèle en utilisant l'optimiseur Adam avec un taux d'apprentissage de 0,001 et une fonction de perte `mse`. Le modèle a été entraîné sur les données d'entraînement en utilisant 50 époques et une taille de lot de 128. Les performances du modèle ont été évaluées sur les données de test.

## Partie 3 : Évaluation du Modèle

### Calcul de l'Erreur

Nous avons implémenté une fonction pour calculer la racine de l'erreur quadratique moyenne (RMSE) entre les données originales et reconstruites. Le RMSE a été calculé sur les ensembles d'entraînement et de test.

### Interprétation des Résultats

- **RMSE sur l'ensemble d'entraînement** : Indique à quel point les données reconstruites par l'autoencodeur sont proches des données originales pour les données d'entraînement.
- **RMSE sur l'ensemble de test** : Indique à quel point les données reconstruites par l'autoencodeur sont proches des données originales pour les données de test.
- **Comparaison des RMSE** : Si le RMSE sur l'ensemble de test est significativement plus élevé que le RMSE sur l'ensemble d'entraînement, cela peut indiquer un surapprentissage.

## Partie 4 : Système de Recommandation

### Extraction des Représentations Encodées

Nous avons utilisé le modèle encodeur pour générer des représentations compactes des textes et affiché quelques exemples de ces représentations.

### Implémentation d'une Fonction de Recommandation

Nous avons créé une fonction `recommend` qui prend en entrée un texte d'utilisateur et le nombre de recommandations souhaitées (par défaut : 5). La fonction :

1. Transforme le texte d'entrée en vecteur TF-IDF.
2. Extrait sa représentation encodée.
3. Calcule la similarité entre cette représentation et celles des textes du dataset.
4. Retourne les textes les plus similaires.

### Test de la Fonction de Recommandation

Nous avons testé la fonction avec un texte exemple : "The stock market shows significant growth". Les recommandations obtenues ont été affichées avec leurs catégories et contenus.

## Conclusion

Ce rapport technique a démontré la construction, l'entraînement et l'évaluation d'un autoencodeur pour la recommandation de textes. Le système de recommandation basé sur les représentations compactes générées par l'autoencodeur a été implémenté et testé avec succès. Les résultats obtenus montrent que l'autoencodeur est capable de reconstruire efficacement les données d'entrée et que le système de recommandation peut suggérer des textes similaires de manière pertinente.
